In [1]:
import pandas as pd
import math
import os
#current working directory for file management
os.getcwd()

'C:\\Users\\woodn\\OneDrive\\Documents\\Online Courses\\UCSD\\work play'

In [2]:
#Options! Modify this cell here to address all of the different flags

#Refurbish motors? set to True if want to include IEEE 1068 Level 2 repair for motors
mtr_flag = True

#build unit specific optionality including strings for input and output files
in_str = './TA_ALKY1&2_ELEC.csv'
out_str = './TA_ALKY1&2_EWL.csv'

In [3]:
#import csv file to dataframe from RCM Analysis for all items with TA Alky1 Label
analysis_df = pd.read_csv(in_str, sep=',')

In [4]:
#setup output dataframe
work_letter_columns = ['Equipment Type','Status','Discipline','Equipment #','Equipment Name','Work Description',
    'Mandatory Item','J Factor','Consequence Description','Conseq Cost','Unmit Prob','Unmit Risk','Mitigated Prob',
    'Mitigated Risk','Direct Cost to Mitigate','Cumulative ROM Cost','In Scope or Out of Scope','Comments','Unit']
ewl_df = pd.DataFrame(columns=work_letter_columns)
new_entry_df = pd.DataFrame(columns=work_letter_columns)

#filter only TA System implementation tasks from list
mask2 = analysis_df['Task Implementation System'] == 'TA System'

if mtr_flag:
    mask3 = 1
else:
    mask3 = ~analysis_df['Task Name'].str.contains('IEEE 1068 Level 2', na=False)
    
#final slicing from masks
analysis_df = analysis_df[mask2 & mask3]

In [5]:
#Create Work Description from all remaining Task List entries
analysis_df['Work Description'] = analysis_df['Task Name'] + ", " + analysis_df['Task Description']

#Discipline lookup dictionary based off SAP Object Type (at a later date add this functionality)
disc_lookup = pd.read_csv('./Disc_Lookup.csv', sep=',').set_index('Object Type').to_dict(orient='dict')['Discipline']

In [6]:
#let's create our initial DataFrame with all TA submittal items by Failure Mode. Need to Harvest Task List and Total POC by Failure Mode and Equip ID first before dropping the duplicates
for eq_id in analysis_df['Equipment'].unique():
    id_mask = analysis_df['Equipment'] == eq_id
    for fm in analysis_df[id_mask]['Failure Mode'].unique():
        fm_mask = analysis_df['Failure Mode'] == fm

        #import/export CSV does not contain Equipment Type column
        new_entry_df['Equipment Type'] = analysis_df[id_mask]['Equipment Type'].unique()
        #Discipline assignment using Object Type
        new_entry_df.at[0,'Discipline'] = disc_lookup[new_entry_df.at[0,'Equipment Type']]
       
        #Assign Unit based off of w_IMS labels assigned to the Analysis
        if analysis_df[id_mask]['Labels'].str.contains('Alky1', na=False).any():
            new_entry_df['Unit'] = 'Alky1'
        elif analysis_df[id_mask]['Labels'].str.contains('Alky2', na=False).any():
            new_entry_df['Unit'] = 'Alky2'
        new_entry_df['Equipment #'] = analysis_df[id_mask & fm_mask]['Equipment'].unique()
        #Equipment Name can be added later

        #Work Description added as a combined string of all Tasks identified (unique on Tasks)
        new_entry_df['Work Description'] = pd.Series(analysis_df[id_mask & fm_mask]['Work Description'].unique()).str.cat(sep='\n')
        #Task Notes added as Comments (unique on Tasks)
        new_entry_df['Comments'] = pd.Series(analysis_df[id_mask & fm_mask]['Task Notes'].unique()).str.cat(sep='\n')
        #Consequence description added from all scenario notes in analysis (unique on FM, common to Tasks)
        new_entry_df['Consequence Description'] = pd.Series(analysis_df[id_mask & fm_mask]['Scenario'].unique()).str.cat(sep='\n')

        #import/export uses Task IsMandatory and normal export uses Task Mandatory
        if pd.Series(analysis_df.columns).str.contains('Task IsMandatory', case=True).any():
            if (analysis_df[id_mask & fm_mask]['Task IsMandatory'] == 'Yes').any():
                new_entry_df['Mandatory Item'] = 'Yes'
            else:
                new_entry_df['Mandatory Item'] = 'No'
        else:
            if (analysis_df[id_mask]['Task Mandatory'] == 'Yes').any():
                new_entry_df['Mandatory Item'] = 'Yes'
            else:
                new_entry_df['Mandatory Item'] = 'No'
        
        #Direct Cost to Mitigate is sum of all task Prices of Conformance (unique to Tasks)
        new_entry_df['Direct Cost to Mitigate'] = analysis_df[id_mask & fm_mask]['POC'].sum()
        #Consequence Cost (unique on FM, common to Tasks)
        new_entry_df['Conseq Cost'] = analysis_df[id_mask & fm_mask]['PONC'].unique().sum()
        
        #Task Annual Factor is being used to determine duration between outages (unique on tasks). 
        #There are some tasks that skip every other outage. This uses the lowest years duration to determine outage window duration. 
        time_w = analysis_df[id_mask & fm_mask]['Task Annual factor'].max()

        #Derived Probability from failure rates and duration between outages.
        ETBF = analysis_df[id_mask & fm_mask]['ETBF (yrs)'].unique().sum()
        new_entry_df['Unmit Prob'] = 1 - math.exp(-1/(ETBF*time_w))
        ETBC = analysis_df[id_mask & fm_mask]['ETBC (yrs)'].unique().sum()
        new_entry_df['Mitigated Prob'] = 1 - math.exp(-1/(ETBC*time_w))

        #Mitigated Risk will be calculated later via DataFrame broadcasting
        ewl_df = ewl_df.append(new_entry_df)

C:\Users\woodn\AppData\Local\Temp\ipykernel_16112\3365174238.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ewl_df = ewl_df.append(new_entry_df)
C:\Users\woodn\AppData\Local\Temp\ipykernel_16112\3365174238.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ewl_df = ewl_df.append(new_entry_df)
C:\Users\woodn\AppData\Local\Temp\ipykernel_16112\3365174238.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ewl_df = ewl_df.append(new_entry_df)
C:\Users\woodn\AppData\Local\Temp\ipykernel_16112\3365174238.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ewl_df = ewl_df.append(new_entry_df)
C:\Users\woodn\AppData\Local\Temp\ipyker

In [7]:
#Unmitigated Risk
ewl_df['Unmit Risk'] = ewl_df['Unmit Prob'] * ewl_df['Conseq Cost']
#Mitigated Risk
ewl_df['Mitigated Risk'] = ewl_df['Mitigated Prob'] * ewl_df['Conseq Cost']
#J-Factor
ewl_df['J Factor'] = (ewl_df['Unmit Risk'] - ewl_df['Mitigated Risk']) / ewl_df['Direct Cost to Mitigate']

In [ ]:
#export work letter to csv file
ewl_df.to_csv(out_str,encoding='utf-8', index=False)